In [1]:
import sagemaker

# Define IAM role
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'arn:aws:iam::975050273734:role/service-role/SageMaker-ExecutionRole-20240612T130383'

In [2]:
import pandas as pd
from io import StringIO

import re

In [3]:
s3 = boto3.client('s3')
bucket = 'capstone-bucket-4-friends'

In [4]:
content = s3.list_objects(Bucket=bucket)["Contents"]
content

[{'Key': 'crsp_stock_2023.csv',
  'LastModified': datetime.datetime(2024, 6, 12, 16, 55, 42, tzinfo=tzlocal()),
  'ETag': '"c513e3fbf4e7477ab6acb9c9f663b567-38"',
  'Size': 641927058,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'caopuzheng',
   'ID': '3e8e9fefba64fe47aada033ac68b6e97b3ca71a1d45879ebba28cd6aa5dcad55'}}]

In [ ]:
def convert_csv_to_parquet(s3, csv_key, parquet_key):
    """
    Downloads a CSV file from S3, converts it to Parquet, and uploads the Parquet file.

    Args:
      csv_key: S3 key of the CSV file.
      parquet_key: S3 key of the output Parquet file.
    """

    # Download CSV file
    s3.download_file(bucket, csv_key, f"/tmp/{csv_key}")

    # Read CSV with Pyarrow
    df = pd.read_csv(f"/tmp/{csv_key}")

    converted_data_parquet = df.to_parquet(parquet_key)

    # Upload Parquet file to S3
    s3.upload_file(converted_data_parquet, bucket, parquet_key)


# Get list of CSV objects in the bucket (with prefix if provided)
paginator = s3.get_paginator("list_objects_v2")
pages = paginator.paginate(Bucket=bucket, Prefix='')

for page in pages:
    for obj in page.get("Contents", []):
        csv_key = obj["Key"]
        # Generate output Parquet key with same filename (replace extension)
        parquet_key = csv_key.replace(".csv", ".parquet")
        convert_csv_to_parquet(s3, csv_key, parquet_key)

print("CSV to Parquet conversion completed!")